In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import os
import bz2
import nltk
from nltk.corpus import stopwords
#from nltk import word_tokenize
from nltk.tokenize import word_tokenize
#libraries for lemmatization
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#nltk.download('averaged_perceptron_tagger')
#nltk.download('all')

In [4]:
os.listdir('/content/drive/MyDrive/sentitment analysis project')

['test.ft.txt.bz2',
 'train.ft.txt.bz2',
 'Train.csv',
 'Test.csv',
 'LTest.csv',
 'LTrain.csv']

In [5]:
train_data = bz2.BZ2File('/content/drive/MyDrive/sentitment analysis project/train.ft.txt.bz2')
test_data  = bz2.BZ2File('/content/drive/MyDrive/sentitment analysis project/test.ft.txt.bz2')

In [6]:
def load_and_extract(data):
    texts, labels = [], []
    for line in data:
        x=line.decode('utf-8') #decodes the binary into strings
        labels.append(int(x[9])-1)
        texts.append(x[10:].strip())
    print('complete')
    return np.array(labels), texts

In [7]:
train_labels, train_texts = load_and_extract(train_data)
test_labels, test_texts = load_and_extract(test_data)

complete
complete


In [8]:
print(f'The length of the texts dataset for training is : {len(train_texts)}')
print(f'The length of the label  dataset for training is : {len(train_labels)}')
print(f'The length of the texts dataset for testing is : {len(test_texts)}')
print(f'The length of the label dataset for testing is : {len(test_labels)}')

The length of the texts dataset for training is : 3600000
The length of the label  dataset for training is : 3600000
The length of the texts dataset for testing is : 400000
The length of the label dataset for testing is : 400000


In [9]:
train_df = pd.DataFrame({'Reviews':train_texts, 'Sentiments':train_labels})
test_df = pd.DataFrame({'Reviews':test_texts, 'Sentiments':test_labels})
test_df.head()

,Reviews,Sentiments
0,Great CD: My lovely Pat has one of the GREAT v...,1
1,One of the best game music soundtracks - for a...,1
2,Batteries died within a year ...: I bought thi...,0
3,"works fine, but Maha Energy is better: Check o...",1
4,Great for the non-audiophile: Reviewed quite a...,1


In [10]:
#check for imbalance in the labels
balance_check = pd.DataFrame(train_df['Sentiments'].value_counts().reset_index())
balance_check.columns = ['sentiments', 'frequency']
balance_check['sentiments'] = balance_check['sentiments'].map({1:'positive', 0:'negative'})
balance_check

,sentiments,frequency
0,positive,1800000
1,negative,1800000


In [11]:
#reduce the dataset for training to avoid the cost of computatiion
train_df1 = train_df[train_df['Sentiments']==1] # rows with positive sentiments
train_df2 = train_df[train_df['Sentiments']==0]   #rows with negative sentiments
#concatenate the two data with positive and negative sentiment
train_dff = pd.concat([train_df1.sample(n=15000), train_df2.sample(n=15000)])
train_dff = train_dff.sample(frac=1)
train_dff.reset_index(inplace=True)
train_dff.drop(columns='index', inplace=True)
train_dff.head()


,Reviews,Sentiments
0,Not just fluff: Get into the ins and outs of P...,1
1,Holding the bipolar spouse & partner responsib...,1
2,works: Works well with the Firstrax barrel/hol...,0
3,Unstable and a little dangerous: This toy does...,0
4,Poorly written for underachievers!: This book ...,0


In [12]:
train_dff['Sentiments'].value_counts()

1    15000
0    15000
Name: Sentiments, dtype: int64

In [13]:
#reduce the dataset for training to avoid the cost of computatiion
test_df1 = test_df[test_df['Sentiments']==1] # rows with positive sentiments
test_df2 = test_df[test_df['Sentiments']==0]   #rows with negative sentiments
#concatenate the two data with positive and negative sentiment
test_dff = pd.concat([test_df1.sample(n=2500), train_df2.sample(n=2500)])
test_dff = test_dff.sample(frac=1)
test_dff.reset_index(inplace=True)
test_dff.drop(columns='index', inplace=True)
test_dff.head()

,Reviews,Sentiments
0,Disapppointed in Amazon: I ordered this produc...,0
1,There is no need to praise this book.: Those w...,1
2,Nancy is wonderful!!!: I must listen to this a...,1
3,"Best beginners' manual: I've used the ""Sailing...",1
4,really really BAD BOOK: The Everyday Grace is ...,0


In [14]:
print(f'the reduced training set is :{train_dff.shape}')
print(f'the reduced testing set is :{test_dff.shape}')

the reduced training set is :(30000, 2)
the reduced testing set is :(5000, 2)


In [15]:
train_dff.to_csv('/content/drive/MyDrive/sentitment analysis project/Train.csv')

In [16]:
test_dff.to_csv('/content/drive/MyDrive/sentitment analysis project/Test.csv')